In [1]:
# Data import
import pandas as pd

df=pd.read_csv("BankChurners.csv",skipinitialspace=True)

df.drop(["CLIENTNUM","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"],axis=1,inplace=True)

# Make target value binary
df = df.replace(
    {
        "Attrition_Flag": {
            "Existing Customer": 0,
            "Attrited Customer": 1,
        }
    }
)



# Feature selection for modeling
#X = df.drop(columns=["Attrition_Flag"])
X = df[["Total_Trans_Ct","Total_Trans_Amt"]]
#X = df[["Total_Trans_Ct","Total_Trans_Amt", "Total_Revolving_Bal", "Total_Amt_Chng_Q4_Q1", "Total_Ct_Chng_Q4_Q1", "Total_Relationship_Count", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Customer_Age", "Credit_Limit", "Months_on_book"]]

# Selecting the "Attrition_Flag" column as the target
y = df["Attrition_Flag"]

X

,Total_Trans_Ct,Total_Trans_Amt
0,42,1144
1,33,1291
2,20,1887
3,20,1171
4,28,816
...,...,...
10122,117,15476
10123,69,8764
10124,60,10291
10125,62,8395


In [2]:
# eXtreme Gradient Boost Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing the xgboost model
xgb = xgb.XGBClassifier(
    learning_rate=0.05,
    max_depth=4,
    min_child_weight=1,
    n_estimators=500,
    subsample=0.4331229815476772,
    colsample_bytree=0.5242723167060505,
    reg_lambda=7.049263425549493e-06,
    reg_alpha=0.015123845878697514,
)

# Fitting the model to the training data
xgb.fit(X_train, y_train)

# Save the model in bin format
xgb.save_model("model.bin")

# Print model score
print("Score: ", xgb.score(X_test, y_test))

# Making predictions on the test set
y_pred = xgb.predict(X_test)

# Creating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Printing the confusion matrix
print("\n")
print("Confusion Matrix:\n", cm)
print("\n")

# Printing the classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


# Example code to make probability prediction, for this example it's mandatory to retrain the model with two features for X (comment/uncomment on cell above to switch the "X" input)
# Create a new dataframe with Total_Trans_Ct and Total_Trans_Amt to simulate a user input
A = 20 # 20 = High probability to churn, 80 = Low probability to churn
B = 40000
user_input = pd.DataFrame([[A, B]], columns=["Total_Trans_Ct", "Total_Trans_Amt"])

prediction = xgb.predict_proba(user_input)
print("This customer have", round(prediction[0][1]*100),"% chance to churn")


Score:  0.8978282329713722


Confusion Matrix:
 [[1629   70]
 [ 137  190]]


Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94      1699
           1       0.73      0.58      0.65       327

    accuracy                           0.90      2026
   macro avg       0.83      0.77      0.79      2026
weighted avg       0.89      0.90      0.89      2026

This customer have 86 % chance to churn
